In [1]:
import glob, json

import sys
sys.path.append("/scr/ig_pipeline")

from b1k_pipeline.utils import get_targets, parse_name

In [2]:
import pathlib

object_lists = {}
for target in get_targets("combined_unfiltered"):
    data = json.loads((pathlib.Path("/scr/ig_pipeline") / "cad" / target / "artifacts/object_list.json").read_text())
    object_lists[target] = data

In [3]:
from bddl.knowledge_base import *

objs_by_id = {obj.name.split("-")[-1]: obj for obj in Object.all_objects()}

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
import collections

attachment_group_fixes = {
  "cameratripod": "cameratripodF"
}

male_attachments = collections.defaultdict(set)
female_attachments = collections.defaultdict(set)
for target, target_info in object_lists.items():
  for item_name, _, parent in target_info["max_tree"]:
    pn = parse_name(item_name)
    if pn is None:
      continue
    if pn.group("meta_type") != "attachment":
      continue
    attachment_type = pn.group("meta_id")
    # Correct known issues
    if attachment_type in attachment_group_fixes:
      attachment_type = attachment_group_fixes[attachment_type]

    # Validate and parse
    if len(attachment_type) == 0:
      print(f"Missing attachment type on object {parent} in target {target}")
      continue
    if attachment_type[-1] not in "MF":
      print(f"Invalid attachment type {attachment_type} on object {parent} in target {target}")
      continue
    is_attachment_female = attachment_type[-1] == "F"
    attachment_type = attachment_type[:-1]
    if len(attachment_type) == 0:
      print(f"Missing attachment type on object {parent} in target {target}")
      continue

    parent_id = parse_name(parent).group("model_id")
    obj = parent_id  # objs_by_id[parent_id]
    if is_attachment_female:
      female_attachments[attachment_type].add(obj)
    else:
      male_attachments[attachment_type].add(obj)

In [6]:
# Check that every attachment has a male and a female end
print("Missing female end:", set(male_attachments) - set(female_attachments))
print("Missing male end:", set(female_attachments) - set(male_attachments))

Missing female end: set()
Missing male end: set()
